In [63]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from robust_optimization import RobustOptimizer
import copy
import timm
import torch.utils.data as data

print(f"Torch: {torch.__version__}")

env: CUDA_VISIBLE_DEVICES=0
Torch: 1.10.2


In [47]:
# Training settings
batch_size = 32
epochs = 40
lr = 3e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

training_size = 28317 + 3541
IMG_SIZE=260
image_path = '../code/dataset/images.npy'
training_emotion_path = "../code/dataset/emotions_multi.npy"
test_emotion_path = '../code/dataset/emotions_multi.npy'

False


In [60]:
emotions[0]

array([0.4, 0. , 0. , 0.1, 0.3, 0.2, 0. , 0. ])

In [69]:
for data, label in train_loader:
    print(label)
    break

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.9000, 0.1000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2222, 0.0000, 0.0000, 0.7778, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.1000, 0.1000, 0.0000, 0.2000, 0.0000],
        [0.1000, 0.7000, 0.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.9000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000],
        [0.7778, 0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.1111],
        [0.4000, 0.0000, 0.0000, 0.4000, 0.0000, 0.1000, 0.1000, 0.0000],
        [0.3750, 0.0000, 0.0000, 0.0000, 0.5000, 0.1250, 0.0000, 0.0000],
        [0.5000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4286, 0.0000, 0.0000, 0.4286, 0.0000, 0.0000, 0.0000, 0.1429],
        [0.8000, 0.0000, 0.1000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2000, 0.7000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000],
        [0.5556, 0.0000, 0.0000, 0.333

In [64]:
def load_data(image_path, training_emotion_path, subset):
    images = np.load(image_path)        # shape = (35393, 48, 48, 1)
    # images = images/255.0
    images = np.float32(images)
    
    if subset == 'train':
        training_emotions = np.load(training_emotion_path)
        training_emotions = np.float32(training_emotions)
        random_training_emotions = []
        for i in range(training_size):
            random_emotion_index = np.random.choice(8, p=emotions[i]/np.sum(emotions[i]))
            emo_lst = [0, 0, 0, 0, 0, 0, 0, 0]
            emo_lst[random_emotion_index] = 1
            random_training_emotions.append(emo_lst)
        return images[:training_size], np.array(random_training_emotions)
    
    if subset == 'test':
        test_emotions = np.load(test_emotion_path)
        test_emotions = np.float32(test_emotions)
        return images[training_size:], test_emotions[training_size:]

def img_proc(img):
    img = torch.tensor(img)                             # (48, 48, 1)
    img = torch.reshape(img, (1, 48, 48))               # (1, 48, 48)
    img = transforms.Resize([IMG_SIZE, IMG_SIZE])(img)  # (1, 260, 260)
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = img.repeat(3, 1, 1)                           # (3, 260, 260)
    return img

class FERPlusDataset(data.Dataset):
    def __init__(self, image_path, training_emotion_path, subset):
        assert(subset=='train' or subset=='test')
        self.images, self.emotions = load_data(image_path, training_emotion_path, subset)

    def __getitem__(self, index):
        image = self.images[index]
        emotion = self.emotions[index]

        return img_proc(image), emotion

    def __len__(self):
        return len(self.images)

train_loader = torch.utils.data.DataLoader(
    FERPlusDataset(
        image_path,
        training_emotion_path,
        'train'
    ),
    batch_size=batch_size,
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    FERPlusDataset(
        image_path,
        training_emotion_path,
        'test'
    ),
    batch_size=batch_size,
    shuffle=False,
)

def accuracy(output, target):
    batch_size = target.size(0)
    acc = 0
    for i in range(batch_size):
        true = target[i]
        pred = output[i]
        index_max = torch.argmax(pred)
        if true[index_max] == torch.max(true):
            acc += 1
    acc = float(acc)/batch_size
    return acc

In [4]:
def train(model,n_epochs=epochs, learningrate=lr, robust=False):
    # optimizer
    if robust:
        optimizer = RobustOptimizer(filter(lambda p: p.requires_grad, model.parameters()), optim.Adam, lr=learningrate)
    else:
        optimizer=optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learningrate)

    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()
        for data, label in tqdm(train_loader):
            data = data.to(device)
            label = label.to(device)

            output = model(data)
            loss = criterion(output, label)

            if robust:
                #optimizer.zero_grad()
                loss.backward()
                optimizer.first_step(zero_grad=True)
  
                # second forward-backward pass
                output = model(data)
                loss = criterion(output, label)
                loss.backward()
                optimizer.second_step(zero_grad=True)
            else:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            acc = (output.argmax(dim=1) == label.argmax(dim=1)).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in val_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = accuracy(val_output, label)
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += val_loss / len(val_loader)

        print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        #scheduler.step()
    
    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in val_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = accuracy(val_output, label)
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += val_loss / len(val_loader)

        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")

In [5]:
def set_parameter_requires_grad(model, requires_grad):
    for param in model.parameters():
        param.requires_grad = requires_grad

In [7]:
criterion = nn.MSELoss().to(device) 
model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))

model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

In [8]:
set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

  0%|          | 0/996 [00:00<?, ?it/s]

Epoch : 1 - loss : 0.0620 - acc: 0.7173 - val_loss : 0.0299 - val_acc: 0.7647



  0%|          | 0/996 [00:00<?, ?it/s]

Epoch : 2 - loss : 0.0562 - acc: 0.7544 - val_loss : 0.0282 - val_acc: 0.7771



  0%|          | 0/996 [00:00<?, ?it/s]

...

Epoch : 8 - loss : 0.0537 - acc: 0.7701 - val_loss : 0.0337 - val_acc: 0.7667



  0%|          | 0/996 [00:00<?, ?it/s]

Epoch : 9 - loss : 0.0536 - acc: 0.7712 - val_loss : 0.0344 - val_acc: 0.7519



  0%|          | 0/996 [00:00<?, ?it/s]

Epoch : 10 - loss : 0.0541 - acc: 0.7691 - val_loss : 0.0286 - val_acc: 0.7809

Best acc:0.7842342342342346
val_loss : 0.0304 - val_acc: 0.7842



In [ ]:
set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

In [ ]:
# PATH='enet_8859.pt'
# torch.save(model, PATH)

In [ ]:
# # Load
# print(PATH)
# model = torch.load(PATH)
# model.eval()